In [21]:
import pandas
import datetime

site_id = 6

weather = pandas.read_csv(f"site_{site_id}_weather.csv", sep=";", parse_dates=['Timestamp'])
consumption = pandas.read_csv(f"site_{site_id}_consumption.csv", sep=";", parse_dates=['Timestamp'])

def datetime_to_epoch(d1):
    """
    January 1st, 1970 at 00:00:00 UTC is referred to as the Unix epoch
    :param d1: input date
    :return: seconds since unix epoch
    """
    if not d1.tzinfo:
        raise ValueError("date is missing timezone information")

    d2 = datetime.datetime(1970, 1, 1, tzinfo=datetime.timezone.utc)
    time_delta = d1 - d2
    ts = int(time_delta.total_seconds())
    return ts

def to_unix_epoch(row):
    return datetime_to_epoch(row["Timestamp"])

weather = weather[weather["Distance"] < 8]

weather["UnixTS"] = weather.apply(to_unix_epoch, axis=1)
consumption["UnixTS"] = consumption.apply(to_unix_epoch, axis=1)

def find_closest_temperature_at_ts(row):
    timestamp = row["UnixTS"]
    loc = weather["UnixTS"].searchsorted(timestamp)
    if loc < len(weather)-1:
        c0 = weather.iloc[ loc ]
        c1 = weather.iloc[ loc+1 ]
        distance = c1["UnixTS"] - c0["UnixTS"]
        print("YO... ", distance)
        alpha = (timestamp - c0["UnixTS"]) / distance
        t0 = weather.iloc[loc]["Temperature"]
        t1 = weather.iloc[loc+1]["Temperature"]
        return (1-alpha) * t0 + alpha * t1
    
    return weather.iloc[loc]["Temperature"]

consumption["Temperature"] = consumption.apply(find_closest_temperature_at_ts, axis=1)
print(consumption.head(100))


/tmp/ipykernel_9200/3549813689.py:36: RuntimeWarning: divide by zero encountered in scalar divide
  alpha = (timestamp - c0["UnixTS"]) / distance
/tmp/ipykernel_9200/3549813689.py:39: RuntimeWarning: invalid value encountered in scalar add
  return (1-alpha) * t0 + alpha * t1
/tmp/ipykernel_9200/3549813689.py:36: RuntimeWarning: invalid value encountered in scalar divide
  alpha = (timestamp - c0["UnixTS"]) / distance


     obs_id  SiteId                 Timestamp  ForecastId         Value   
0   1231308       6 2013-01-01 01:00:00+00:00          43  25108.373290  \
1   1855136       6 2013-01-01 01:15:00+00:00          43  25062.047878   
2   5379308       6 2013-01-01 01:30:00+00:00          43  25015.722466   
3   1204858       6 2013-01-01 01:45:00+00:00          43  24969.397055   
4    167176       6 2013-01-01 02:00:00+00:00          43  24923.071643   
..      ...     ...                       ...         ...           ...   
95  1584855       6 2013-01-02 00:45:00+00:00          43  25177.861408   
96  4414228       6 2013-01-02 01:00:00+00:00          43  25201.024114   
97  7679920       6 2013-01-02 01:15:00+00:00          43  24946.234349   
98  3636173       6 2013-01-02 01:30:00+00:00          43  24691.444583   
99  2507416       6 2013-01-02 01:45:00+00:00          43  24784.095407   

        UnixTS  Temperature  
0   1357002000          NaN  
1   1357002900          NaN  
2   13570